# 0. Import Library

In [1]:
import os
if not os.path.exists("./tfdet"):
    !git clone -q http://github.com/burf/tfdetection.git
    !mv ./tfdetection/tfdet ./tfdet
    !rm -rf ./tfdetection

In [2]:
#ignore warning
import warnings, os
warnings.filterwarnings(action = "ignore")
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
import tensorflow as tf
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

import tfdet

# 1. Build Detector

In [3]:
import numpy as np

image_shape = [320, 320]
n_class = 21
batch_size = 1
proposal_count = 100
iou_threshold = 0.5
score_threshold = 0.25

sample_data = np.random.random([batch_size, *image_shape, 3]).astype(np.float32)

In [4]:
save_path = "./model.h5"

x = tf.keras.layers.Input(shape = [*image_shape, 3], batch_size = batch_size) #init batch_size for tensorrt and tflite's map_fn
out = tfdet.model.detector.effdet_lite_d0(x, n_class = n_class, scale = [24, 48, 96, 192, 320], ratio = [0.5, 1, 2], octave = 3, weights = None)
nms_out = tfdet.model.postprocess.effdet.FilterDetection(proposal_count = proposal_count, iou_threshold = iou_threshold, score_threshold = score_threshold)(out)
trt_out = tfdet.model.postprocess.effdet.FilterDetection(tensorrt = True)(out) # for add tensorrt nms
model = tf.keras.Model(x, nms_out)
trt_model = tf.keras.Model(x, trt_out)
try:
    model.load_weights(save_path) #load weight
except:
    pass

[p.shape for p in model.predict(sample_data, verbose = 0)], [p.shape for p in trt_model.predict(sample_data, verbose = 0)]

([(1, 100, 21), (1, 100, 4)], [(1, 19206, 21), (1, 19206, 4)])

# 2. TF2TF-Lite

2-1. Convert

In [4]:
save_path = "./model.tflite"
tfdet.export.tf2lite(model, save_path, dtype = tf.float32)

'./model.tflite'

2-2. Load

In [5]:
save_path = "./model.tflite"
convert_model = tfdet.export.load_tflite(save_path)
pred = convert_model(sample_data)
del convert_model
[p.shape for p in pred]

[(1, 100, 21), (1, 100, 4)]

# 3. TF2ONNX

3-1. Convert

In [6]:
save_path = "./model.onnx"
tfdet.export.tf2onnx(model, save_path, opset = 13)

'./model.onnx'

3-2. Load

In [7]:
convert_model = tfdet.export.load_onnx(save_path, gpu = 0)
pred = convert_model(sample_data)
del convert_model
[p.shape for p in pred]

[(1, 100, 21), (1, 100, 4)]

# 4. TF2TensorRT

3-1. convert(tf > onnx > add_trt_nms > tensorrt)

In [8]:
save_path = "./trt_model.onnx"
tfdet.export.tf2onnx(trt_model, save_path, opset = 13)

'./trt_model.onnx'

In [9]:
save_nms_path = "./trt_model_nms.onnx"
#raw_output > [num_dets, det_bboxes, det_scores, det_classes], not raw_output > [det_bboxes, det_logits]
tfdet.export.add_trt_nms(save_path, save_nms_path, proposal_count = proposal_count, iou_threshold = iou_threshold, score_threshold = score_threshold, raw_output = True)

'./trt_model_nms.onnx'

In [10]:
save_trt_path = "./trt_model_nms.trt"
tfdet.export.onnx2trt(save_nms_path, save_trt_path, dtype = "FP32", memory_limit = 1)

'./trt_model_nms.trt'

3-2. load

In [11]:
convert_model = tfdet.export.load_trt(save_trt_path)
pred = convert_model(sample_data)
del convert_model
[p.shape for p in pred]

[(1, 1), (1, 100, 4), (1, 100), (1, 100)]